# CADS API Python client Tests

In [ ]:
import os
import xarray as xr

import cads_api_client

In [ ]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.copernicus-climate.eu/api")

## Client instantiation

The client expose the APIs for both 
- catalogue exploration 
- data retrieve

In [ ]:
client = cads_api_client.ApiClient(api_url)
client

## 1. Catalogue Exploration

**Objective**: verify the capabilities of the client to access the catalogue and to expose the catalogue information

### 1.1 Collections

**Objective**: Verify the access to the list of **collections** and thier descriptions

_**expected result**: correct instatiation of collections object._
*if collections.response has 200 status code, the request is succesfull and the test can proceed*

In [ ]:
collections = client.collections()
collections.response

_**expected result**: list of all available collection._

In [ ]:
collections.collection_ids()

### 1.2 Collection

**Objective**: Verify the access to the list of **collection** and it description

_**expected result**: correct instatiation of collection object._

*if collection.response has 200 status code, the request is succesfull and the test can proceed*

In [ ]:
# select a collection
collection = client.collection("reanalysis-era5-pressure-levels")
collection

_**expected result**: display end datetime of the collection._

In [ ]:
collection.end_datetime()

_**expected result**: json of the collection response describing the collection._

In [ ]:
collection.json

## 2. Data Retrieve

 **Objective**: verify the capabilities of the client to retrieve a data


The retrieve can be done using an high level function that perform the submission, the monitoring and the download:
- _client.retrieve_ see **2.1**

or it can be done using the low level api (see **3** Data Retrieve: advanced usage): 
- client.sumbit for submitting the request, see **3.1**
- remote.status for monitoring the request, see **3.2**
- remote.download for downloading the result, see **3.3**


### 2.1  Retrieve: small data request

The **retrieve** function is blocking: 
- submits the request
- waits until the requests is completed
- downloads the data


_**expected result**: the client subit the requests and when the process is completed, the client downloads file._

In [ ]:
output_path = client.retrieve(
    collection_id="reanalysis-era5-pressure-levels",
    target="test01.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month="01", 
    day="25", 
    time="06:00"
)

output_path

In [ ]:
ls test01.grib

_**expected output**: data compatible with the requests:_
- _one variable: temperature (t)_
- _two dimensions latitude and longitude_
- _time coordinate has lenght 1_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 3 Data Retrieve: adavanced API

**Objective:** Verify the capability of the client to retreive the data using the advanced API 

### 3.1 Request Submission: big data request

**client.submit** function is async and returns a remote object that allows the monitoring of the request status

_**expected result**: the client subit the requests and return a remote object that allows the process monitoring._

In [ ]:
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="test02.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["1971"], 
    month=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    day=[
        '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
        '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
        '25', '26', '27', '28', '29', '30', '31',
    ],
    time="06:00"
)
remote.request_uid

### 2.3 Request Monitoring

**remote.status** allows to monitor the process status

_**expected result**: return the updated stuatus of the requests: failed, successful, running_

In [ ]:
remote.status

**client.get_requests** returns the list of requests submitted 

_**expected result**: ids of the submitted requests._

In [ ]:
requests = client.get_requests()
requests.job_ids()

_**expected result**: the remote.request_uid is in the list of the submitted requests._

In [ ]:
remote.request_uid in requests.job_ids()

it is possibile to recreate the remote object starting from the requests id

_**expected result**: new remote that allow to monitor the request._

In [ ]:
status_info = client.get_request(remote.request_uid)
new_remote = status_info.make_remote()
new_remote

_**expected result**: new remote id is equal to the orginal id._

In [ ]:
new_remote.request_uid == remote.request_uid

### 2.4 Data download

**download** function it blocking: 
- waits until the requests is completed
- downloads the data

In [ ]:
output_path = remote.download("test02.grib")
output_path

_**expected output**: data compatible with the requests:_
- _one variable: temperature (t)_
- _three dimensions time (365), latitude (721) and longitude (1440)_

In [ ]:
ds = xr.open_dataset(output_path)
ds

## 4. Error Handling

**Objectives**: Verify how error are handled: error messages and codes

### 4.1 Wrong URL

_**expected output**: ConnectionError: HTTPConnectionPool

In [ ]:
client = cads_api_client.ApiClient("http://wrong_url/api")
client.collections()

### 4.2 Missing collection

_**expected output**: 404 Client Error: Not Found for url_

In [ ]:
client = cads_api_client.ApiClient(api_url)
client.collection("missing_collection")

### 4.3 Unknown job

_**expected output**: 404 Client Error: Not Found for url_

In [ ]:
status_info = client.get_request("ffffffff-4455-6677-8899-aabbccddeeff")

### 4.4 Wrong request

**async request using collection.submit**

_**expected output**: _collection.submit doesn't raise any Error_


In [ ]:
client = cads_api_client.ApiClient(api_url)
collection = client.collection("reanalysis-era5-pressure-levels")
remote = collection.submit( 
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["2222"], 
    month="01", 
    day="25", 
    time="06:00"
)

_**expected output**: remote.wait_on_result raises a ProcessingFailedError and dispalys the information of the json result (see next cells) in the format: {type} - {title} - {detail}. Where:_
-  _type is RuntimeError_ 
- _title is job failed_
- _detail contains the error traceback_

In [ ]:
remote.wait_on_result()

_**expected output**: result status code is 400_

In [ ]:
results = remote.make_results()

In [ ]:
results.status_code

_**expected output**:_
- _result.json describes the error in the fields: "type", "title" and "detail"_
- _"detail" contains the error traceback_

In [ ]:
results.json

**sync request using collection.retrieve**

_**expected output**: client.retrieve raises a ProcessingFailedError and dispalys the information of the json result in the format: {type} - {title} - {detail}. Where:_
-  _type is RuntimeError_ 
- _title is job failed_
- _detail contains the error traceback_


In [ ]:
remote = client.retrieve( 
    "reanalysis-era5-pressure-levels",
    target="output.grib", # optional
    product_type="reanalysis", 
    format="grib", 
    variable="temperature", 
    pressure_level="1", 
    year=["2222"], 
    month="01", 
    day="25", 
    time="06:00"
)